In [104]:
import json
import openai
import nest_asyncio
from typing import Dict
from pathlib import Path
from instructor import patch
from pydantic import BaseModel, Field
from pydantic.json import pydantic_encoder
patch()
nest_asyncio.apply()

In [105]:
story_name = 'one-day-prisoner'
story_path = Path('../prompts/stories')
result_path = Path('../results')
instruction_path = Path('../prompts/instructions')

In [121]:
model_name = 'gpt-3.5-turbo-16k'
max_tokens = 2048
stop = None
temperature = 0.5
language = 'Chinese'

In [107]:
story_path = story_path / f'{story_name}.txt'
character_prompt_path = instruction_path / 'character.txt'
storyboard_prompt_path = instruction_path / 'storyboard.txt'
character_result_path = result_path / f'{story_name}_character.txt'
storyboard_result_path = result_path / f'{story_name}_storyboard.txt'

In [110]:
def load_text(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def save_text(response, output_file):
    with open(output_file, 'w') as file:
        file.write(response + '\n')

In [111]:
def call_openai_api(history, 
                    content,
                    model_name: str='gpt-3.5-turbo-16k',
                    max_tokens: int=2048,
                    stop: str=None,
                    temperature: float=0.5):
    
    history.append({'role': 'user', 
                    'content': content})
    
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=history,
        max_tokens=max_tokens,
        stop=stop,
        temperature=temperature,
    )
    
    return response.choices[0]['message']['content'].strip()

In [112]:
def split_into_chunks(text, tokens=500):
    chunks = []
    
    for i in range(0, len(text), tokens):
        chunks.append(text[i:i + tokens])
        
    return chunks

In [134]:
def process_story(story_path, 
                  character_prompt_path,
                  storyboard_prompt_path,
                  model_name: str='gpt-3.5-turbo-16k',
                  max_tokens: int=4096,
                  stop: str=None,
                  temperature: float=0.5,
                  language: str='Chinese'):
    
    story = load_text(story_path)
    chunks = split_into_chunks(story)
    
    # Initialize the conversation history
    history = [{'role': 'system', 
                'content': 'You are a helpful assistant.'},
               {'role': 'system',
                'content': 'Below is the long text provided by the user.'}]
    
    if language != 'English':
        history.append({'role': 'system',
                        'content': f'Please translate all your response in {language}'})
    
    # Add each chunk as a user message to history
    for chunk in chunks:
        history.append({'role': 'user', 
                        'content': chunk})

    character_prompt = load_text(character_prompt_path)
    storyboard_prompt = load_text(storyboard_prompt_path)
    
    print('The model is getting the summary.')
    
    # Task 1: Summarize characters
    character_summary = call_openai_api(history, character_prompt, model_name, 
                                        max_tokens, stop, temperature)
    
    # Update the history to include the character summary
    history.append({'role': 'assistant', 
                    'content': character_summary})
    
    print('The model is getting the storyboard script.')
    
    # Task 2: Generate storyboard script
    storyboard_script = call_openai_api(history, storyboard_prompt, model_name, 
                                        max_tokens, stop, temperature)
    
    # Save to output file
    return character_summary, storyboard_script

In [135]:
character_summary, storyboard_script = process_story(story_path, 
                                                     character_prompt_path,
                                                     storyboard_prompt_path,
                                                     model_name,
                                                     max_tokens,
                                                     stop, 
                                                     temperature,
                                                     language)

The model is getting the summary.
The model is getting the storyboard script.


In [130]:
character_summary, storyboard_script = process_story(story_path, 
                                                     character_prompt_path,
                                                     storyboard_prompt_path,
                                                     model_name,
                                                     max_tokens,
                                                     stop, 
                                                     temperature,
                                                     language)

The model is getting the summary.
The model is getting the storyboard script.


In [138]:
save_text(storyboard_script, storyboard_result_path)
save_text(character_summary, character_result_path)

In [137]:
print(character_summary)

{
  'B先生': {
    'Name': 'B先生',
    'High-Level Synopsis': 'B先生是一个被困在时间循环中的人，每天都重复着同样的一天。他的存在对于故事的发展和主题非常重要，引发了关于时间、困境和自我意识的思考。',
    'Appearance and Attire': {
      'Physical Features': '苍白而腼腆的面容，戴眼镜，光头',
      'Clothing': '黑衣'
    },
    'Psychological Profile': 'B先生表现出腼腆、愁苦、渴望和绝望的情绪。他在困境中经历了情绪的起伏，从最初的好奇和探索，到后来的愤怒、绝望和疯狂。',
    'Socio-Cultural Context': '没有提供足够的信息',
    'Relationships': {
      '管理员': '与管理员有几次互动，管理员成为他唯一的交流对象'
    },
    'Key Moments': {
      '租房': 'B先生租下了一间小套房，开始了他的时间囚徒生活',
      '循环突破尝试': 'B先生试图打破时间循环，进行了一系列实验和行动，包括故意改变行为和观察他人的反应',
      '心灵的思考': 'B先生在困境中开始思考时间和自我意识的问题，对自己的存在和世界的本质有了更深层次的思考'
    },
    'Additional Nuances': {
      '时间囚徒': 'B先生被困在时间循环中，每天重复着同样的一天，他的存在和行为受到时间的限制和影响'
    }
  },
  '管理员': {
    'Name': '管理员',
    'High-Level Synopsis': '管理员是大楼里的管理人员，成为B先生唯一的交流对象。他在B先生的困境中起到了观察者和支持者的角色。',
    'Appearance and Attire': {
      'Physical Features': '没有提供足够的信息',
      'Clothing': '没有提供足够的信息'
    },
    'Psychological Profile': '管理员表现出关心和好奇的态度，对B先生的行为和困境感到惊

In [136]:
print(storyboard_script)

{
  'Scene 1': {
    'Text-to-Image Prompt': 'B先生租下小套房，黑衣男子陪同，管理员办理手续。',
    'Camera Angle': 'Wide shot',
    'Environment': 'Interior of the building, a small room with minimal furniture',
    'Characters Present': 'B先生, 黑衣男子, 管理员',
    'Visual Design': 'Dimly lit room, sparse furniture, a desk with paperwork',
    'Dialogue': 'B先生: "我想租一间不带家具的房子。" 管理员: "请填写这份租约。"',
    'Sound Elements': 'Subtle background music, low volume',
    'Emotional Atmosphere': 'Tense, mysterious'
  },
  'Scene 2': {
    'Text-to-Image Prompt': 'B先生抱怨房子的视野狭窄。',
    'Camera Angle': 'Medium shot',
    'Environment': 'Interior of the small room',
    'Characters Present': 'B先生, 黑衣男子',
    'Visual Design': 'Foldable canvas wardrobe, inflatable mattress, limited space',
    'Dialogue': 'B先生: "房子的视野太狭窄了。" 黑衣男子: "这是你们最短的租期吗？"',
    'Sound Elements': 'Silence',
    'Emotional Atmosphere': 'Uneasy, claustrophobic'
  },
  'Scene 3': {
    'Text-to-Image Prompt': 'B先生对管理员表示感谢，并请他离开。',
    'Camera Angle': 'Close-up',
   